# GGIR use case 1

This use case requires the generation of three signals: rest, sleep and main bed period. All signals appear in a time signal between 12:00 noon and 12:00 next day. Each signal consists of two binary signals: period starts and period ends. For reference, there is a larger [description of this use case](https://github.com/sequgen/sequgen/issues/21) as well as a [visualization](https://cran.r-project.org/web/packages/GGIR/vignettes/GGIR.html#42_Output_part_4) of the time signals (click on the tab `4.2.3 visualisation_sleep.pdf`).

## 1. Time axis

We model the time axis as a discrete list of minutes starting at 12\*60 and ending at 36\*60

In [1]:
import numpy as np

time_axis = np.arange(12*60, 36*60+1)

## 2. Channels

We start with a zero signal

In [2]:
from sequgen.deterministic.constant import constant

channel_1 = constant(time_axis, 0)

Next, we define how many peaks we want, of what width and at what location

In [3]:
import random
from sequgen.dimension import Dimension
from sequgen.parameter_space import ParameterSpace

def overlap(peaks):
    peaks = [ peak for peak in sorted(peaks, key=lambda p:p['location']) ]
    for i in range(0, len(peaks)-1):
        if peaks[i]['location'] + peaks[i]['width'] >= peaks[i+1]['location']:
            return(True)
    return(False)

iterations = random.randint(8,12)
peaks = []
start_time = time_axis[0]
while len(peaks) < iterations:
    channel_1_parameter_space = ParameterSpace([
        Dimension("location", start_time, time_axis[-1]),
        Dimension("width", 10, 30),
    ])
    new_peak = channel_1_parameter_space.sample()
    if not overlap(peaks+[new_peak]):
        peaks.append(new_peak)

In [4]:
peaks

[{'location': 844.6832038089898, 'width': 28.86811004628262},
 {'location': 1607.0707313223427, 'width': 27.832839252549945},
 {'location': 1258.2611848841843, 'width': 17.969290975131415},
 {'location': 1824.0410289912827, 'width': 25.339087800995205},
 {'location': 1478.172539982449, 'width': 27.763902801091426},
 {'location': 1674.2902318654558, 'width': 21.796208634427515},
 {'location': 2138.1054655357575, 'width': 15.316048374888132},
 {'location': 1073.2816828555506, 'width': 28.992778326933664},
 {'location': 1281.2940788829344, 'width': 11.007700940398312},
 {'location': 1428.6736924064255, 'width': 19.743754138923432}]